# BigData Capstone

## Group: U_Data

    Members:
    
    Aroob Almutairi 
    
    Abdulrahman Alosaimi
    
    Duaa Al saad 
    
    Fatimah Alateeq
    
    Ghaliah mohammedhussein


## Project describtion

We build 4 models of each classification and regression. We used salary as our target in classification, and capital gain in regression.

## Pre-Processing

1. Replaced the '?' in all the columns with Not-specified
2. Made the values for education consistent
3. Changed the values for salary to be 0 if <=50k, and 1 if >50k
4. Sumed the values for the fnlwgt column, and get the percentage of fnlwgt
5. Found the difference between capital gain and loss
6. Found the workholic people

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
#lets import our usual suspects
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

#pyspark related imports
import pyspark
from pyspark.sql.functions import col, isnan, when, count
from pyspark.ml.feature import Imputer, VectorAssembler, StringIndexer
from pyspark.sql.functions import *
from pyspark.sql import functions as f
from pyspark.ml.regression import RandomForestRegressor, DecisionTreeRegressor, GBTRegressor, LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, LogisticRegression, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from pyspark.ml import Pipeline

In [3]:
spark = pyspark.sql.SparkSession.builder.config("spark.executor.memory", "8g").config("spark.driver.memory", "8g").getOrCreate()

22/12/26 14:14:03 WARN Utils: Your hostname, ghalyts-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.13 instead (on interface en0)
22/12/26 14:14:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/26 14:14:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## PySpark

In [4]:
# Load the dataset.
dfs = spark.read.csv('Cleaned_Data.csv',inferSchema=True,header=False)

In [6]:
# The data Shape
print(dfs.count(), len(dfs.columns))
dfs.printSchema()

1000000 15
root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: integer (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital_gain: integer (nullable = true)
 |-- capital_loss: integer (nullable = true)
 |-- hours_per_week: integer (nullable = true)
 |-- native_country: string (nullable = true)
 |-- salary: string (nullable = true)



In [7]:
dfs.show(5)

+---+----------------+------+---------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
|age|       workclass|fnlwgt|education|education_num|    marital_status|       occupation| relationship| race|gender|capital_gain|capital_loss|hours_per_week|native_country|salary|
+---+----------------+------+---------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
| 39|       State-gov| 77516|Bachelors|           13|     Never-married|     Adm-clerical|Not-in-family|White|  Male|        2174|           0|            40| United-States| <=50K|
| 50|Self-emp-not-inc| 83311|Bachelors|           13|Married-civ-spouse|  Exec-managerial|      Husband|White|  Male|           0|           0|            13| United-States| <=50K|
| 38|         Private|215646|  HS-grad|            9|          Divorced|Handlers-cleaners|Not-i

# MODEL

In [38]:
#This step will label encode all the categorical columns and store them in different columns with the same name + '_idx', 
#so category will become category_idx 
cat_cols = ['workclass', 'education', 'marital_status','occupation','relationship','race','gender','native_country']

for c in cat_cols: 
    indexer = StringIndexer(inputCol=c, outputCol=c+'_idx') 
    dfs = indexer.fit(dfs).transform(dfs) 
    
final_df = dfs.drop(*cat_cols) 

In [39]:
final_df.show()

+---+------+-------------+------------+------------+--------------+------+-----------------+---------------------------+------+----------+-------------+-------------+------------------+--------------+----------------+--------+----------+------------------+
|age|fnlwgt|education_num|capital_gain|capital_loss|hours_per_week|salary|fnlwgt_percentage|diffrence_capital_gain_loos|profit|workaholic|workclass_idx|education_idx|marital_status_idx|occupation_idx|relationship_idx|race_idx|gender_idx|native_country_idx|
+---+------+-------------+------------+------------+--------------+------+-----------------+---------------------------+------+----------+-------------+-------------+------------------+--------------+----------------+--------+----------+------------------+
| 39| 77516|           13|        2174|           0|            40|     0|              0.0|                       2174|     1|         0|          4.0|          2.0|               1.0|           5.0|             1.0|     0.0|   

In [40]:
# dfs=dfs.where(dfs.native_country!='South')
final_df = final_df.withColumn("fnlwgt_percentage", dfs["fnlwgt_percentage"].cast('int'))
final_df = final_df.withColumn("workclass_idx", dfs["workclass_idx"].cast('int'))
final_df = final_df.withColumn("education_idx", dfs["education_idx"].cast('int'))
final_df = final_df.withColumn("marital_status_idx", dfs["marital_status_idx"].cast('int'))
final_df = final_df.withColumn("occupation_idx", dfs["occupation_idx"].cast('int'))
final_df = final_df.withColumn("relationship_idx", dfs["relationship_idx"].cast('int'))

final_df = final_df.withColumn("race_idx", dfs["race_idx"].cast('int'))
final_df = final_df.withColumn("gender_idx", dfs["gender_idx"].cast('int'))
final_df = final_df.withColumn("native_country_idx", dfs["native_country_idx"].cast('int'))


## Classification model

In [41]:
cols = final_df.columns # Extract the column names from the dataframe
cols.remove('salary') #remove salary because it is our target in clasification 

#vector assembler will take all the columns and convert them into one column called features
assembler = VectorAssembler(inputCols=cols, outputCol='features')
final_df = assembler.transform(final_df)

In [42]:
final_df.show(5,truncate=False)

22/12/26 14:17:17 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+---+------+-------------+------------+------------+--------------+------+-----------------+---------------------------+------+----------+-------------+-------------+------------------+--------------+----------------+--------+----------+------------------+--------------------------------------------------------------------------------------+
|age|fnlwgt|education_num|capital_gain|capital_loss|hours_per_week|salary|fnlwgt_percentage|diffrence_capital_gain_loos|profit|workaholic|workclass_idx|education_idx|marital_status_idx|occupation_idx|relationship_idx|race_idx|gender_idx|native_country_idx|features                                                                              |
+---+------+-------------+------------+------------+--------------+------+-----------------+---------------------------+------+-----

#### Lets Split

In [43]:
# We will now create a new dataframe only consisting of the features column and the label column
df_data = final_df.select(col('features'), col('salary').alias('label'))

#simple data splitting
df_train, df_test = df_data.randomSplit([0.7, 0.3])

### Model Building
Our data is now ready for some model building

In [44]:
# Decision Tree
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")
model_dt = dt.fit(df_train)

In [45]:
# Random Forest
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)
model_rf = rf.fit(df_train)

In [46]:
# Logistic Regression
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, labelCol="label", featuresCol="features")
model_lr = lr.fit(df_train)

22/12/26 14:19:04 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/12/26 14:19:04 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [47]:
# Gradient Boost
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)
model_gbt = gbt.fit(df_train)

In [49]:
# Lets make the predictions on the testing set
pred_dt = model_dt.transform(df_test)
pred_rf = model_rf.transform(df_test)
pred_lr = model_lr.transform(df_test)
pred_gbt = model_gbt.transform(df_test)


### Evaluation

In [50]:
# Accuracy Metric
evaluator_A = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

# F1 Metric
evaluator_F = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

# Weighted Precision
evaluator_P = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")

# Weighted Recall
evaluator_R = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")

# Our models
models = [pred_dt, pred_rf, pred_lr, pred_gbt]

# Empty lists that will store the scores for each metric for each model.
accuracy = []
F1 = []
precision = []
recall = []

# loop to populate the empty lists with scores of models for each metric.
for model in models:
    accuracy.append(evaluator_A.evaluate(model))
    F1.append(evaluator_F.evaluate(model))
    precision.append(evaluator_P.evaluate(model))
    recall.append(evaluator_R.evaluate(model))

In [51]:
# We will convert all lists created above into a dataframe for easy viewing.
df_ev = pd.DataFrame(list(zip(accuracy, F1, precision, recall)), 
                     columns = ['Accuracy', 'F1-Score', 'Weighted Precision', 'Weighted Recall'],
                     index = ['Decision Tree', 'Random Forest', 'Logistic Regression', 'Gradient Boost'])

In [52]:
df_ev

,Accuracy,F1-Score,Weighted Precision,Weighted Recall
Decision Tree,0.509883,0.443157,0.510274,0.509883
Random Forest,0.508909,0.495179,0.507634,0.508909
Logistic Regression,0.506525,0.340609,0.256568,0.506525
Gradient Boost,0.511259,0.505039,0.510369,0.511259


### Hyperparameter Tuning - Best Two Models

Choosing Decision Tree and Gradient Boosted Trees instead of Linear Regression because of the overfitting

In [53]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel

#### Decision Tree

In [54]:
#initialize our grid 
grid2 = ParamGridBuilder().addGrid(dt.maxDepth, [5, 10, 15, 20 , 25]).build()

In [55]:
#CrossValidator
cv_reg2 = CrossValidator(estimator=dt, estimatorParamMaps=grid2, evaluator=evaluator_A, parallelism=2, numFolds=5)

In [56]:
# fit on training set
cv_reg2Model = cv_reg2.fit(df_train)

22/12/26 14:29:53 WARN DAGScheduler: Broadcasting large task binary with size 1054.3 KiB
22/12/26 14:29:54 WARN DAGScheduler: Broadcasting large task binary with size 1054.3 KiB
22/12/26 14:29:54 WARN DAGScheduler: Broadcasting large task binary with size 1382.6 KiB
22/12/26 14:29:54 WARN DAGScheduler: Broadcasting large task binary with size 1382.6 KiB
22/12/26 14:29:55 WARN DAGScheduler: Broadcasting large task binary with size 1806.1 KiB
22/12/26 14:29:55 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB
22/12/26 14:29:56 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
22/12/26 14:29:57 WARN DAGScheduler: Broadcasting large task binary with size 1113.6 KiB
22/12/26 14:29:57 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB
22/12/26 14:29:58 WARN DAGScheduler: Broadcasting large task binary with size 4.7 MiB


22/12/26 14:29:59 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB
22/12/26 14:30:05 WARN DAGScheduler: Broadcasting large task binary with size 1054.3 KiB
22/12/26 14:30:06 WARN DAGScheduler: Broadcasting large task binary with size 1382.6 KiB
22/12/26 14:30:07 WARN DAGScheduler: Broadcasting large task binary with size 1806.1 KiB
22/12/26 14:30:07 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB
22/12/26 14:30:08 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
22/12/26 14:30:09 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB
22/12/26 14:30:11 WARN DAGScheduler: Broadcasting large task binary with size 4.7 MiB


22/12/26 14:30:12 WARN DAGScheduler: Broadcasting large task binary with size 6.0 MiB


22/12/26 14:30:14 WARN DAGScheduler: Broadcasting large task binary with size 7.4 MiB


22/12/26 14:30:16 WARN DAGScheduler: Broadcasting large task binary with size 9.1 MiB


...

22/12/26 14:33:30 WARN DAGScheduler: Broadcasting large task binary with size 10.9 MiB


22/12/26 14:33:31 WARN DAGScheduler: Broadcasting large task binary with size 13.2 MiB


22/12/26 14:33:34 WARN DAGScheduler: Broadcasting large task binary with size 15.7 MiB


22/12/26 14:33:36 WARN DAGScheduler: Broadcasting large task binary with size 10.5 MiB


22/12/26 14:34:19 WARN DAGScheduler: Broadcasting large task binary with size 1107.8 KiB
22/12/26 14:34:19 WARN DAGScheduler: Broadcasting large task binary with size 1507.8 KiB
22/12/26 14:34:19 WARN DAGScheduler: Broadcasting large task binary with size 2044.0 KiB
22/12/26 14:34:20 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
22/12/26 14:34:20 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
22/12/26 14:34:21 WARN DAGScheduler: Broadcasting large task binary with size 4.7 MiB
22/12/26 14:34:22 WARN DAGScheduler: Broadcasting large task binary with size 6.2 MiB


In [57]:
# Accuracy Metric
evaluator_A.evaluate(cv_reg2Model.transform(df_test))

22/12/26 14:34:24 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB


0.5104290120188941

In [58]:
#average metrics on 4 different models 
cv_reg2Model.avgMetrics 

[0.5111982426011041,
 0.5110948248851509,
 0.511050237858857,
 0.5123282059493637,
 0.5113149416538245]

#### Gradient Boosted Trees 

In [59]:
#initialize our grid 
grid = ParamGridBuilder().addGrid(gbt.maxIter, [40, 30, 15, 20 , 25]).build()

In [60]:
#CrossValidator
cv_reg = CrossValidator(estimator=gbt, estimatorParamMaps=grid, evaluator=evaluator_A, parallelism=2, numFolds=5)

In [61]:
# fit on training set
cv_regModel = cv_reg.fit(df_train)

22/12/26 14:35:36 WARN BlockManager: Block rdd_3225_0 already exists on this machine; not re-adding it


In [62]:
# Accuracy Metric
evaluator_A.evaluate(cv_regModel.transform(df_test))

0.510227501716253

In [63]:
#average metrics on 4 different models 
cv_regModel.avgMetrics 

[0.5112795685286986,
 0.5109166269336128,
 0.5111943914651279,
 0.5110155481331214,
 0.5112882614779906]

### Pipeline - Best Two Models

In [64]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import PCA

#### Decision Tree

In [65]:
# create a pipeline containing PCA and Decision Tree
pca = PCA(k=5, inputCol='features', outputCol='pcaFeature')
dt_1 = DecisionTreeClassifier(maxDepth=10)
pipeline = Pipeline (stages=[pca, dt_1])

In [66]:
# fit on training set
pipeline_model = pipeline.fit(df_train)

22/12/26 14:43:40 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/12/26 14:43:40 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [67]:
# Accuracy Metric
evaluator_R.evaluate(pipeline_model.transform(df_test))

0.5127378419271216

#### Gradient Boosted Trees 

In [68]:
# create a pipeline containing PCA and Gradient Boosted Trees 
pca1 = PCA(k=5, inputCol='features', outputCol='pcaFeature')
GBTR_1 = GBTClassifier(maxIter = 20)
pipeline2 = Pipeline (stages=[pca1, GBTR_1])

In [69]:
# fit on training set
pipeline_model2 = pipeline2.fit(df_train)

In [70]:
# Accuracy Metric
evaluator_R.evaluate(pipeline_model2.transform(df_test))

0.5105383057423605

## Regression Model 

In [74]:
final_df2 = dfs.drop(*cat_cols) 
# dfs=dfs.where(dfs.native_country!='South')
final_df2 = final_df2.withColumn("fnlwgt_percentage", dfs["fnlwgt_percentage"].cast('int'))
final_df2 = final_df2.withColumn("workclass_idx", dfs["workclass_idx"].cast('int'))
final_df2 = final_df2.withColumn("education_idx", dfs["education_idx"].cast('int'))
final_df2 = final_df2.withColumn("marital_status_idx", dfs["marital_status_idx"].cast('int'))
final_df2 = final_df2.withColumn("occupation_idx", dfs["occupation_idx"].cast('int'))
final_df2 = final_df2.withColumn("relationship_idx", dfs["relationship_idx"].cast('int'))

final_df2 = final_df2.withColumn("race_idx", dfs["race_idx"].cast('int'))
final_df2 = final_df2.withColumn("gender_idx", dfs["gender_idx"].cast('int'))
final_df2 = final_df2.withColumn("native_country_idx", dfs["native_country_idx"].cast('int'))

In [75]:
cols = final_df2.columns 
cols.remove('capital_gain') #remove capital_gain because it is our target in regression 

#vector assembler will take all the columns and convert them into one column called features
assembler = VectorAssembler(inputCols=cols, outputCol='features')
assembler_df = assembler.transform(final_df2)

In [76]:
# We will now create a new dataframe only consisting of the features column and the label column 
df_reg = assembler_df.select(col('features'), col('capital_gain').alias('label'))

#simple data splitting
df_train, df_test = df_reg.randomSplit([0.7, 0.3])

### Build the models

In [77]:
#Linear Regression
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
r_lr = lr.fit(df_train)

In [78]:
#Random Forest Regressor
rf = RandomForestRegressor(featuresCol='features', labelCol='label')
r_rf = rf.fit(df_train)

In [79]:
#Gradient Boosted Tree Regressor
gbt = GBTRegressor(featuresCol="features", labelCol='label', maxIter=10)
r_gbt = gbt.fit(df_train)

In [80]:
#Decision Tree Regressor
dt = DecisionTreeRegressor(featuresCol="features", labelCol='label')
r_dt = dt.fit(df_train)

In [81]:
# Lets make the predictions on the testing set
pred_dt = r_dt.transform(df_test)
pred_rf = r_rf.transform(df_test)
pred_lr = r_lr.transform(df_test)
pred_gbt = r_gbt.transform(df_test)

### Evaluate the models

In [82]:
evaluator = RegressionEvaluator()

In [83]:
#list our regrassion models
models = [pred_rf, pred_gbt, pred_dt, pred_lr]

evaluator_R = RegressionEvaluator(predictionCol='prediction', labelCol='label', metricName='r2')

evaluator_RMSE = RegressionEvaluator(predictionCol='prediction', labelCol='label', metricName='rmse')

evaluator_MAE = RegressionEvaluator(predictionCol='prediction', labelCol='label', metricName='mae')

# Empty lists that will store the scores for each metric for each model.
R2 = []
RMSE = []
MAE = []

# loop to populate the empty lists with scores of models for each metric.
for model in models:
    R2.append(evaluator_R.evaluate(model))
    RMSE.append(evaluator_RMSE.evaluate(model))
    MAE.append(evaluator_MAE.evaluate(model))

In [84]:
# We will convert all lists created above into a dataframe for easy viewing.
df_ev_reg = pd.DataFrame(list(zip(R2, RMSE, MAE)), 
                     columns = ['R-squared', 'Root Mean Squared Error', 'Mean Absolute Error'],
                     index = ['Random Forest Regressor', 'Gradient Boosted Trees Regressor', 'Decision Tree Regressor', 'Linear Regression'])

In [85]:
df_ev_reg

,R-squared,Root Mean Squared Error,Mean Absolute Error
Random Forest Regressor,0.728885,5662.905996,2180.794710
Gradient Boosted Trees Regressor,0.786523,5025.027490,1083.505837
Decision Tree Regressor,0.780244,5098.392648,1379.815423
Linear Regression,1.000000,0.396031,0.242107


### Hyperparameter Tuning - Best Two Models

Choosing Decision Tree and Gradient Boosted Trees instead of Linear Regression because of the overfitting

In [86]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel

#### Decision Tree

In [87]:
#initialize our grid 
grid2 = ParamGridBuilder().addGrid(dt.maxDepth, [5, 10, 15, 20 , 25]).build()

In [88]:
#CrossValidator
cv_reg2 = CrossValidator(estimator=dt, estimatorParamMaps=grid2, evaluator=evaluator_R, parallelism=2, numFolds=5)

In [89]:
# fit on training set
cv_reg2Model = cv_reg2.fit(df_train)

22/12/26 14:54:52 WARN DAGScheduler: Broadcasting large task binary with size 1475.9 KiB
22/12/26 14:54:53 WARN DAGScheduler: Broadcasting large task binary with size 1475.9 KiB
22/12/26 14:54:53 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
22/12/26 14:54:53 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
22/12/26 14:54:54 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB
22/12/26 14:54:55 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
22/12/26 14:54:55 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB
22/12/26 14:54:55 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
22/12/26 14:54:56 WARN DAGScheduler: Broadcasting large task binary with size 7.7 MiB


22/12/26 14:54:57 WARN DAGScheduler: Broadcasting large task binary with size 1300.1 KiB
22/12/26 14:54:58 WARN DAGScheduler: Broadcasting large task binary with size 11.1 MiB


22/12/26 14:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1657.0 KiB


22/12/26 14:55:01 WARN DAGScheduler: Broadcasting large task binary with size 15.4 MiB


22/12/26 14:55:02 WARN DAGScheduler: Broadcasting large task binary with size 2000.7 KiB


...

22/12/26 15:00:29 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB


22/12/26 15:00:32 WARN DAGScheduler: Broadcasting large task binary with size 45.4 MiB


22/12/26 15:00:34 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB


22/12/26 15:00:37 WARN DAGScheduler: Broadcasting large task binary with size 30.3 MiB


22/12/26 15:00:39 WARN DAGScheduler: Broadcasting large task binary with size 30.3 MiB


In [90]:
# Accuracy Metric
evaluator_R.evaluate(cv_reg2Model.transform(df_test))

0.7802437582084216

In [91]:
#average metrics on 4 different models 
cv_reg2Model.avgMetrics 

[0.7720932742789746,
 0.757863172093168,
 0.6773178566972267,
 0.5834472480828815,
 0.5484308788230956]

#### Gradient Boosted Trees 

In [92]:
#initialize our grid 
grid = ParamGridBuilder().addGrid(gbt.maxIter, [40, 30, 15, 20 , 25]).build()

In [93]:
#CrossValidator
cv_reg = CrossValidator(estimator=gbt, estimatorParamMaps=grid, evaluator=evaluator_R, parallelism=2, numFolds=5)

In [94]:
# fit on training set
cv_regModel = cv_reg.fit(df_train)

In [95]:
# Accuracy Metric
evaluator_R.evaluate(cv_regModel.transform(df_test))

0.7869387457164968

In [96]:
#average metrics on 4 different models 
cv_regModel.avgMetrics 

[0.7704864171768898,
 0.770883392964161,
 0.7707110220329045,
 0.7708912026089141,
 0.7708921394838875]

### Pipeline - Best Two Models

In [97]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import PCA

#### Decision Tree

In [98]:
# create a pipeline containing PCA and Decision Tree
pca = PCA(k=5, inputCol='features', outputCol='pcaFeature')
dt_1 = DecisionTreeRegressor(maxDepth=10)
pipeline = Pipeline (stages=[pca, dt_1])

In [99]:
# fit on training set
pipeline_model = pipeline.fit(df_train)

In [100]:
# Accuracy Metric
evaluator_R.evaluate(pipeline_model.transform(df_test))

0.7674569146092332

#### Gradient Boosted Trees 

In [101]:
# create a pipeline containing PCA and Gradient Boosted Trees 
pca1 = PCA(k=5, inputCol='features', outputCol='pcaFeature')
GBTR_1 = GBTRegressor(maxIter = 20)
pipeline2 = Pipeline (stages=[pca1, GBTR_1])

In [102]:
# fit on training set
pipeline_model2 = pipeline2.fit(df_train)

In [103]:
# Accuracy Metric
evaluator_R.evaluate(pipeline_model2.transform(df_test))

0.7868486082923655

# THANK YOU :)